# **Mount**

Mount the google drive directory

In [ ]:
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install opencv-python==4.4.0.46
!pip install import-ipynb

     |████████████████████████████████| 49.5MB 87kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=4d935281e65bafe4bef6a5671cd08b5c6c84d3901b89fed2a1285ccdf8fdffd9
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


# **Unzip**
Unzip the database if is not yet has been done

In [ ]:
'''
# Unzipping dataset
%cd '/content/gdrive/MyDrive/progetto/dbs/' 
!unzip -q facial_expression.zip
''' 

"\n# Unzipping dataset\n%cd '/content/gdrive/MyDrive/progetto/dbs/' \n!unzip -q facial_expression.zip\n"

In [ ]:
# %cp '/content/gdrive/MyDrive/progetto/dbs/mirflickr25k' '/content/gdrive/MyDrive/progetto/dbs/dataset/Training/

# **Model and Initialization**

Import the preconstructed model(facenet_keras) as well as initialize variables

In [ ]:
import glob
import os
import numpy as np
import PIL

import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.python.keras.preprocessing import dataset_utils
from IPython.display import display
from tqdm.notebook import tqdm

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC

from keras.models import load_model

model = load_model('/content/gdrive/MyDrive/progetto/modello/keras-facenet/model/facenet_keras.h5')
#model.load_weights('/content/gdrive/MyDrive/progetto/modello/keras-facenet/weights/facenet_keras_weights.h5')

np.random.seed(42)
tf.random.set_seed(42)

BASE_DIR = '/content/gdrive/MyDrive/progetto/dbs/facial_expression/dataset'
#BASE_DIR = '/content/gdrive/MyDrive/progetto/dbs/dataset'
TEST_DATA_DIR = os.path.join(BASE_DIR, 'PublicTest')
TRAIN_DATA_DIR = os.path.join(BASE_DIR, 'Training')

BATCH_SIZE = 256

# **Dataset**
For the dataset we collect all the files using the available function of keras image_dataset_from_directory(); by using shuffle=False we mantain the alphanumerical order. We obtain 3589 files divided by 7 classes(suprise,neutral,sad,happy,fear,angry,disgust).
# **Retriving image paths,labels,classes and image ids**
Using the utility function index_directory() we obtain the image_paths,labels(the numerical label corrisponding to each classes) of each .jpg in the dataset and class_names. In addition, we collect all the ids by taking the terminal part of each path

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    seed=123,
    shuffle=False,
    image_size=(160, 160),
    batch_size=BATCH_SIZE,
    color_mode='rgb')

testing_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DATA_DIR,
    seed=123,
    shuffle=False,
    image_size=(160, 160),
    batch_size=BATCH_SIZE,
    color_mode='rgb')

testing_image_paths, testing_labels, testing_class_names = dataset_utils.index_directory(TEST_DATA_DIR, labels='inferred', formats='.jpg', shuffle=False)
testing_image_ids = [path.split('/')[10] for path in testing_image_paths]
#image_paths, labels, class_names = dataset_utils.index_directory(TRAIN_DATA_DIR, labels='inferred', formats='.jpg', shuffle=False)
#image_ids = [path.split('/')[9] for path in image_paths]


Found 28709 files belonging to 7 classes.
Found 3589 files belonging to 7 classes.
Found 3589 files belonging to 7 classes.


# **Extracting features**
We will use the preconstructed model, by calling its predict() function,to extract the features from the dataset

In [ ]:
'''
#testing_features = model.predict(testing_dataset,batch_size=BATCH_SIZE,verbose=1)
features = model.predict(dataset,batch_size=BATCH_SIZE, verbose=1)
'''

113/113 [==============================] - 8628s 76s/step


# **Extracting features from model tuning**
We will use the model fine-tuned, by calling its predict() function,to extract the features from the dataset

In [ ]:
'''
model_finetune = tf.keras.models.load_model('/content/gdrive/MyDrive/progetto/modello/keras-facenet/model/fine_tuning_all_unfreezed.h5')
model_finetune = tf.keras.Model(inputs=model_finetune.input, outputs=model_finetune.get_layer('classifier_hidden_tun').output)

#fine_tuned_features = model_finetune.predict(dataset,batch_size=BATCH_SIZE, verbose=1)


testing_features_tuned = model_finetune.predict(testing_dataset,batch_size=BATCH_SIZE,verbose=1)
'''

15/15 [==============================] - 1463s 97s/step


# **Extracting features from another model tuning**
We will use the model fine-tuned, by calling its predict() function,to extract the features from the dataset

In [ ]:
'''
model_finetune_new = tf.keras.models.load_model('/content/gdrive/MyDrive/progetto/modello/keras-facenet/model/fine_tuning_epochs_2.h5')
model_finetune_new = tf.keras.Model(inputs=model_finetune_new.input, outputs=model_finetune_new.get_layer('classifier_hidden').output)

#fine_tuned_features_new = model_finetune_new.predict(dataset,batch_size=BATCH_SIZE, verbose=1)


testing_features_tuned_new = model_finetune_new.predict(testing_dataset,batch_size=BATCH_SIZE,verbose=1)
'''

15/15 [==============================] - 1068s 71s/step


 # **Extracting features from another model tuning with early stopping**
We will use the model fine-tuned, by calling its predict() function,to extract the features from the dataset

In [ ]:
model_finetune_early_stop = tf.keras.models.load_model('/content/gdrive/MyDrive/progetto/modello/keras-facenet/model/fine_tuning_nuovo.h5')
model_finetune_early_stop = tf.keras.Model(inputs=model_finetune_early_stop.input, outputs=model_finetune_early_stop.get_layer('classifier_hidden_tun').output)

#fine_tuned_features_early_stop = model_finetune_early_stop.predict(dataset,batch_size=BATCH_SIZE, verbose=1)


testing_features_tuned_early_stop = model_finetune_early_stop.predict(testing_dataset,batch_size=BATCH_SIZE,verbose=1)

15/15 [==============================] - 1175s 78s/step


# **Saving as numpy 1**
We save the ids, features and labels of the images into pickles

In [ ]:
import sklearn
from sklearn import preprocessing
'''
np.save('/content/gdrive/MyDrive/progetto/pickles/image_ids.npy', image_ids)
np.save('/content/gdrive/MyDrive/progetto/pickles/image_features.npy', features)
np.save('/content/gdrive/MyDrive/progetto/pickles/image_features_normalized.npy', sklearn.preprocessing.normalize(features))
np.save('/content/gdrive/MyDrive/progetto/pickles/image_labels.npy', labels)

#np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_ids.npy', testing_image_ids)
#np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_features.npy', testing_features)
#np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_features_normalized.npy', sklearn.preprocessing.normalize(testing_features))
#np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_labels.npy', testing_labels)

'''

In [ ]:
'''
np.save('/content/gdrive/MyDrive/progetto/pickles/image_paths.npy', image_paths)
#np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_paths.npy', testing_image_paths)

'''

# **Saving as numpy 2**
We save the ids, features and labels of the images into pickles

In [ ]:
'''
import sklearn
from sklearn import preprocessing


#np.save('/content/gdrive/MyDrive/progetto/pickles/image_ids_finetuning.npy', image_ids)
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_features_finetuning.npy', fine_tuned_features)
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_features_normalized_finetuning.npy', sklearn.preprocessing.normalize(fine_tuned_features))
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_labels_finetuning.npy', labels)

np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_ids_finetuning.npy', testing_image_ids)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_features_finetuning.npy', testing_features_tuned)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_features_normalized_finetuning.npy', sklearn.preprocessing.normalize(testing_features_tuned))
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_labels_finetuning.npy', testing_labels)
'''

In [ ]:
'''
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_paths_finetuning.npy', image_paths)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_paths_finetuning.npy', testing_image_paths)
'''

# **Saving as numpy 3**
We save the ids, features and labels of the images into pickles

In [ ]:
'''
import sklearn
from sklearn import preprocessing


#np.save('/content/gdrive/MyDrive/progetto/pickles/image_ids_finetuning_new.npy', image_ids)
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_features_finetuning_new.npy', fine_tuned_features_new)
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_features_normalized_finetuning_new.npy', sklearn.preprocessing.normalize(fine_tuned_features_new))
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_labels_finetuning_new.npy', labels)

np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_ids_finetuning_new.npy', testing_image_ids)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_features_finetuning_new.npy', testing_features_tuned_new)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_features_normalized_finetuning_new.npy', sklearn.preprocessing.normalize(testing_features_tuned_new))
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_labels_finetuning_new.npy', testing_labels)

'''

In [ ]:
'''
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_paths_finetuning_new.npy', image_paths)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_paths_finetuning_new.npy', testing_image_paths)
'''

# **Saving as numpy 4**
We save the ids, features and labels of the images into pickles

In [ ]:
import sklearn
from sklearn import preprocessing


#np.save('/content/gdrive/MyDrive/progetto/pickles/image_ids_finetuning_early_stop.npy', image_ids)
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_features_finetuning_early_stop.npy', fine_tuned_features_early_stop)
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_features_normalized_finetuning_early_stop.npy', sklearn.preprocessing.normalize(fine_tuned_features_early_stop))
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_labels_finetuning_early_stop.npy', labels)

np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_ids_finetuning_early_stop.npy', testing_image_ids)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_features_finetuning_early_stop.npy', testing_features_tuned_early_stop)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_features_normalized_finetuning_early_stop.npy', sklearn.preprocessing.normalize(testing_features_tuned_early_stop))
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_labels_finetuning_early_stop.npy', testing_labels)

In [ ]:
#np.save('/content/gdrive/MyDrive/progetto/pickles/image_paths_finetuning_early_stop.npy', image_paths)
np.save('/content/gdrive/MyDrive/progetto/pickles/testing_image_paths_finetuning_early_stop.npy', testing_image_paths)